# CUDASTF 教程 Part 1: 简介与入门

## 1. CUDASTF 简介 (Sequential Task Flow, STF)

CUDASTF 是一个为 CUDA 实现的**顺序任务流 (STF)** 模型。它通过自动化数据传输和隐式数据依赖，帮助开发者高效地开发多 GPU 并行应用。

- 仅头文件 C++ 库，基于 CUDA API
- 自动推断任务依赖，生成有向无环图 (DAG)
- 支持自动并行化和透明数据管理

### 依赖规则
| 依赖类型 | 说明 |
|---|---|
| 写后读 (RAW) | 任务需等待前序写完成后才能读 |
| 读后写 (WAR) | 任务需等待前序读完成后才能写 |
| 写后写 (WAW) | 两个写同一数据的任务需串行 |
| 并发读 | 多个只读任务可并发 |

这些规则应用于串行算法后，自动生成任务的 DAG，CUDASTF 利用该 DAG 进行并发调度。


## 2. CUDASTF 的安装与使用

- CUDASTF 属于 CCCL 项目，不随 CUDA Toolkit 分发，可在 [CCCL GitHub](https://github.com/NVIDIA/cccl) 获取。确保您已经克隆了 `cccl` 仓库到 `tutorial` 目录的同级 (即 `stf_exp/cccl`)。
- 主要头文件：`#include <cuda/experimental/stf.cuh>`
- 命名空间：`cuda::experimental::stf`

### 编译示例
您可以在本 notebook 所在的目录 (`tutorial/stf/`) 找到示例代码文件，如 `01-axpy.cu` 和 `axpy-annotated.cu`。您可以直接打开这些文件进行修改。

使用 nvcc (推荐用于单个文件编译测试)：
```sh
nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/01-axpy.cu -o 01-axpy -arch=sm_86 -lcuda
```
* `-I../cccl/libcudacxx/include -I../cccl/cudax/include`: 指定 `libcudacxx` 和 `cudax` (包含 STF) 的头文件路径。这个路径是相对于当前 notebook (`tutorial/stf/`) 的。 
* `01-axpy.cu`: 您的源文件名。
* `-o 01-axpy`: 输出的可执行文件名。
* `-arch=sm_86`: 根据您的 GPU 型号指定计算能力。`sm_86` 对应 NVIDIA Ampere 架构 (如 RTX 30系列)。如果您的GPU型号不同，请修改为相应的计算能力，例如 `sm_75` (Turing), `sm_70` (Volta), `sm_61` (Pascal)。您可以通过 `nvidia-smi` 查看GPU型号，并通过NVIDIA官方文档查询其计算能力。
* `-lcuda`: 链接 CUDA runtime 库。

使用 CMake (推荐方式，已在 `tutorial/stf/CMakeLists.txt` 中配置，适用于更复杂的项目)：
```sh
# 确保您在 stf_exp 目录下，并且已经克隆了 cccl 仓库
cd ../../ # 从 tutorial/stf/ 返回到 stf_exp/
mkdir -p build && cd build
cmake .. 
make cudax.cpp17.example.stf.01-axpy # 根据 CMakeLists.txt 中的目标名编译
# 编译成功后，可执行文件通常在 stf_exp/build/examples/stf/ 目录下，例如：
# ./examples/stf/cudax.cpp17.example.stf.01-axpy 
```


## 3. AXPY 示例详解

AXPY (Y = Y + αX) 是线性代数中的基础操作。下面是用 CUDASTF 实现的完整示例。您可以打开本目录下的 [`01-axpy.cu`](./stf/01-axpy.cu) 文件查看和修改。


In [ ]:
// 01-axpy.cu (位于当前 notebook 目录下: tutorial/stf/01-axpy.cu)
// 您可以直接修改此文件并重新运行下面的编译和执行 cell
#include <cuda/experimental/stf.cuh>
#include <cassert> // 包含 cassert 以使用 assert
#include <cmath>   // 包含 cmath 以使用 std::fabs, std::sin, std::cos
#include <cstdio>  // 包含 cstdio 以使用 printf

using namespace cuda::experimental::stf;

// CUDA Kernel for AXPY operation
__global__ void axpy_kernel(double a, slice<const double> x, slice<double> y) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int nthreads = gridDim.x * blockDim.x;
  for (int i = tid; i < x.size(); i += nthreads) {
    y(i) += a * x(i);
  }
}

// Host functions to initialize data
double X0(int i) { return std::sin((double)i); } 
double Y0(int i) { return std::cos((double)i); }

int main() {
  // 1. Create a CUDASTF context
  context ctx;

  // 2. Prepare host data
  const size_t N = 16; // Size of vectors
  double X[N], Y[N];
  for (size_t i = 0; i < N; i++) { 
    X[i] = X0(i); 
    Y[i] = Y0(i); 
  }
  double alpha = 3.14; // Scalar alpha value

  // 3. Create logical data handles for STF
  // STF will manage data movement between host and device
  auto lX = ctx.logical_data(X); // Wraps host array X
  auto lY = ctx.logical_data(Y); // Wraps host array Y

  // 4. Submit a task to the context
  // lX.read(): X is read-only in this task
  // lY.rw(): Y is read-write in this task
  ctx.task(lX.read(), lY.rw())->*[&](cudaStream_t s, auto dX, auto dY) {
    // This lambda executes on the host and enqueues the CUDA kernel
    // s: CUDA stream provided by STF for this task
    // dX, dY: device-side slice objects for X and Y, managed by STF
    axpy_kernel<<<16, 128, 0, s>>>(alpha, dX, dY);
  };

  // 5. Finalize the context
  // This blocks until all submitted tasks are complete and data is synchronized back to host if needed.
  ctx.finalize();

  // 6. Verify results on the host
  for (size_t i = 0; i < N; i++) {
    assert(std::fabs(Y[i] - (Y0(i) + alpha * X0(i))) < 0.0001);
    assert(std::fabs(X[i] - X0(i)) < 0.0001); // X should remain unchanged
  }
  printf("01-axpy: Result is correct!\n");
  
  return 0;
}


### 编译与运行 (在 Notebook 的 Cell 中执行)
您可以在下面的 cell 中直接编译和运行 `01-axpy.cu`。请确保该文件保存在当前目录且与上面的代码块内容一致。


In [2]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/01-axpy.cu -o 01-axpy -arch=sm_86 -lcuda
!./01-axpy

### 生成任务依赖图
在终端中 (或 notebook cell 中加 `!` ) 执行以下命令：
```sh
CUDASTF_DOT_FILE=axpy.dot ./01-axpy
dot -Tpng axpy.dot -o axpy.png
```
生成的 `axpy.png` 即为任务流依赖图。您可以使用 `![axpy_graph](axpy.png)` 在 markdown cell 中显示图片 (如果图片已生成在 notebook 同目录下，且 notebook信任该 markdown)。


In [3]:
!CUDASTF_DOT_FILE=axpy.dot ./01-axpy
!dot -Tpng axpy.dot -o axpy.png

![axpy_graph](axpy.png)

## 4. 进阶：带注释的 AXPY 示例

可以为 `logical_data` 和 `task` 添加 `set_symbol("name")`，便于调试和可视化。您可以打开本目录下的 [`axpy-annotated.cu`](./stf/axpy-annotated.cu) 文件查看和修改。


In [ ]:
// axpy-annotated.cu (位于当前 notebook 目录下: tutorial/stf/axpy-annotated.cu)
// 您可以直接修改此文件并重新运行下面的编译和执行 cell
#include <cuda/experimental/stf.cuh>
#include <cassert> // 包含 cassert 以使用 assert
#include <cmath>   // 包含 cmath 以使用 std::fabs, std::sin, std::cos
#include <cstdio>  // 包含 cstdio 以使用 printf

using namespace cuda::experimental::stf;

// CUDA Kernel for AXPY operation (same as before)
__global__ void axpy_kernel_annotated(double a, slice<const double> x, slice<double> y) {
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int nthreads = gridDim.x * blockDim.x;
  for (int i = tid; i < x.size(); i += nthreads) {
    y(i) += a * x(i);
  }
}

// Host functions to initialize data
double X0_annotated(int i) { return std::sin((double)i); } 
double Y0_annotated(int i) { return std::cos((double)i); }

int main() {
  context ctx;
  const size_t N = 16;
  double X[N], Y[N];
  for (size_t i = 0; i < N; i++) { 
    X[i] = X0_annotated(i); 
    Y[i] = Y0_annotated(i); 
  }
  double alpha = 3.14;

  // Create logical data handles with symbols for visualization
  auto lX = ctx.logical_data(X).set_symbol("X_Input_Data");
  auto lY = ctx.logical_data(Y).set_symbol("Y_InputOutput_Data");

  // Submit a task with a symbol for visualization
  ctx.task(lX.read(), lY.rw()).set_symbol("AXPY_Computation_Task")->*[&](cudaStream_t s, auto dX, auto dY) {
    axpy_kernel_annotated<<<16, 128, 0, s>>>(alpha, dX, dY);
  };

  ctx.finalize();

  // Verify results
  for (size_t i = 0; i < N; i++) {
    assert(std::fabs(Y[i] - (Y0_annotated(i) + alpha * X0_annotated(i))) < 0.0001);
    assert(std::fabs(X[i] - X0_annotated(i)) < 0.0001);
  }
  printf("axpy-annotated: Result is correct!\n");
  
  return 0;
}


### 编译与运行 (在 Notebook 的 Cell 中执行)
您可以直接在下面的 cell 中编译和运行 [`axpy-annotated.cu`](./stf/axpy-annotated.cu)。


In [4]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/axpy-annotated.cu -o axpy-annotated -arch=sm_86 -lcuda
!./axpy-annotated

### 生成带 symbol 的任务图
```sh
CUDASTF_DOT_FILE=axpy_annotated.dot ./axpy-annotated
dot -Tpng axpy_annotated.dot -o axpy_annotated.png
```

你可以用图片查看工具打开 [`axpy_annotated.png`](./axpy-annotated.png)，观察任务和数据的 symbol 标注。
*(如果图片已生成，可以在此 markdown cell 中使用 `![axpy_annotated_graph](axpy_annotated.png)` 显示)*



In [5]:
!CUDASTF_DOT_FILE=axpy_annotated.dot ./axpy-annotated
!dot -Tpng axpy_annotated.dot -o axpy_annotated.png

![axpy_annotated_graph](axpy_annotated.png)